In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from dataclasses import dataclass

from lsst.daf.butler import Butler
import lsst.afw.image as afwImage
import lsst.geom as geom
from astroML.crossmatch import crossmatch_angular
from lsst.daf.butler import Butler
import numpy as np
from astropy.table import Table
from lsst.geom import Point2D
from lsst.pipe.tasks.calibrate import CalibrateTask
from lsst.pipe.tasks.characterizeImage import CharacterizeImageTask
from lsst.source.injection.inject_exposure import ExposureInjectTask
from lsst.meas.extensions.psfex.psfexPsfDeterminer import PsfexNoGoodStarsError
import h5py
import concurrent.futures
from multiprocessing import Lock, Semaphore, Manager
from astropy.io import ascii
import os
from multiprocessing import Value, Lock
import logging
import cv2

In [29]:
repo = "/repo/main"
collection = "LSSTComCam/runs/DRP/DP1/w_2025_17/DM-50530"
where = "instrument='LSSTComCam' AND skymap='lsst_cells_v1' AND day_obs>=20241101 AND day_obs<=20241127 AND exposure.observation_type='science' AND band in ('u','g','r','i','z','y') AND (exposure not in (0))"
#where = "instrument='LSSTComCam' AND skymap='lsst_cells_v1' AND exposure.observation_type='science'"
butler = Butler(repo, collections=collection)
refs = list(set(butler.registry.queryDatasets("preliminary_visit_image", where=where, instrument="LSSTComCam", findFirst=True)))
refs = sorted(refs, key=lambda r: str(r.dataId["visit"]*1000+r.dataId["detector"]))

In [2]:
import numpy as np
import pandas as pd
from lsst.daf.butler import Butler

repo = "/repo/main"
collection = "LSSTComCam/runs/DRP/DP1/w_2025_17/DM-50530"
where = "instrument='LSSTComCam' AND skymap='lsst_cells_v1' AND day_obs>=20241101 AND day_obs<=20241127 AND exposure.observation_type='science' AND band in ('u','g','r','i','z','y') AND (exposure not in (2024110600163, 2024112400111, 2024110800318, 2024111200185, 2024111400039, 2024111500225, 2024111500226, 2024111500239, 2024111500240, 2024111500242, 2024111500288, 2024111500289, 2024111800077, 2024111800078, 2024112300230, 2024112400094, 2024112400225, 2024112600327))"
butler = Butler(repo, collections=collection)
refs = list(set(butler.registry.queryDatasets("preliminary_visit_image", where=where, instrument="LSSTComCam", findFirst=True)))
refs = sorted(refs, key=lambda r: str(r.dataId["visit"]*1000+r.dataId["detector"]))

bad = []

# Only test pairs that exist in refs
pairs = sorted({(int(r.dataId["visit"]), int(r.dataId["detector"])) for r in refs})

for i, (visit, detector) in enumerate(pairs, 1):
    print(f"\rProcessing {i}/{len(pairs)}  bad:{len(bad)}", end="", flush=True)
    try:
        calexp = butler.get(
            "preliminary_visit_image",
            dataId={"instrument": "LSSTComCam", "visit": visit, "detector": detector},
        )
        wcs = calexp.wcs
        if wcs is None:
            bad.append((visit, detector, "wcs_none"))
        else:
            try:
                _ = wcs.getPixelScale()
            except Exception as e:
                bad.append((visit, detector, f"getPixelScale_fail:{type(e).__name__}"))
    except Exception as e:
        bad.append((visit, detector, f"butler_get_fail:{type(e).__name__}"))

print()  # newline

bad_df = pd.DataFrame(bad, columns=["visit", "detector", "reason"])
bad_df.to_csv("bad_pairs.csv", index=False)
print("Wrote bad_pairs.csv with", len(bad_df), "rows")


Processing 24/5634  bad:0

KeyboardInterrupt: 

In [15]:
import pandas as pd

In [20]:
!ls

ADCNN  Notebooks  checkpoints  folders.txt
DATA   README.md  experiments  requirements.txt


In [21]:
train_csv=pd.read_csv("./DATA/train.csv")

In [22]:
train_csv

,ra,dec,source_type,trail_length,mag,beta,visit,detector,integrated_mag,PSF_mag,SNR,physical_filter,x,y,stack_detection,stack_mag,stack_mag_err,stack_snr,image_id
0,59.451460,-48.581017,Trail,55.334405,24.576112,81.244215,2024111900353,6,20.218624,21.427997,6.157103,y,3995,82,True,21.502065,0.180624,6.011046,0
1,59.804769,-48.665247,Trail,31.471505,25.529281,27.887758,2024111900353,6,21.784488,22.712302,1.773262,y,291,2565,False,NaN,NaN,NaN,0
2,59.844418,-48.600055,Trail,32.993556,24.616533,68.452771,2024111900353,6,20.820460,21.771247,4.211660,y,953,3640,False,NaN,NaN,NaN,0
3,59.572675,-48.668341,Trail,50.034901,23.226265,53.464885,2024111900353,6,18.978082,20.136242,19.718109,y,1878,315,True,20.859234,0.103439,10.496388,0
4,59.558339,-48.556756,Trail,17.351005,22.182388,117.693337,2024111900353,6,19.084077,19.735873,28.917499,y,3595,1365,True,19.944823,0.044295,24.511231,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15715,40.201221,-34.347122,Trail,21.709558,23.676156,20.410959,2024112400233,4,20.334529,21.046710,115.010396,r,3741,2691,False,NaN,NaN,NaN,730
15716,40.181249,-34.527696,Trail,21.994242,24.435111,115.028228,2024112400233,4,21.079338,21.797402,46.910170,r,499,3029,False,NaN,NaN,NaN,730
15717,40.207762,-34.487025,Trail,55.351392,26.369343,169.464051,2024112400233,4,22.011522,23.176143,16.876356,r,1293,3273,True,24.270227,0.175228,6.196123,730
15718,40.263298,-34.440469,Trail,7.253646,21.557030,53.070433,2024112400233,4,19.405639,19.697638,410.714299,r,2273,3919,True,19.767826,0.004713,230.383522,730


In [24]:
np.unique(train_csv["image_id"].to_numpy())

array([  0,   2,   4,   7,  10,  11,  12,  16,  17,  21,  22,  23,  24,
        25,  26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,
        38,  39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,
        51,  52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,
        64,  65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,
        77,  78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,
        90,  91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102,
       103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115,
       116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128,
       129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141,
       142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154,
       155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167,
       168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180,
       181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 19